# TD Ameritrade – Python API
The TD Ameritrade API allows users to easily execute trades and pull both fundamental and technical data on all public equity and debt securities.  It also offers data on publicly traded derivatives such as stock options, index & commodity futures, and interest rate swaps.

### Set Up
1.	Go to https://developer.tdameritrade.com/
2.	Click “Register”
3.	Fill in requested fields
4.	Once Registered, click on “My Apps”
5.	Select “Add a new App”
6.	Fill out the fields as follows 
7.	Once the app is approved, click on the app name and navigate to the “KEYS” tab: 
8.	Now that your app is registered copy the “Key” from the above section, and head over to the included Jupyter notebook to test out some of the functionality of the API.

### Python Set up for the API
Now that your app is registered, you will be able to pull a wealth of info from the API.
#### Needed Packages
1.	requests
2.	pandas
3.	time
4.  matplotlib
5.  numpy
6.  seaborn
7.  statsmodels.api
8.  math
9. statistics

#### Needed Configuration Inputs
1.	Ticker file path.  This is a CSV file that you will create that has the set of tickers you wish to analyze.  Place this file in your download's folder.
2.	API Key: pulled from the developer page
3.	Define these required configurations in the Jupyter notebook:
  

### Data & Analysis

1. Downloading fundamental data
2. Downloading pricing data
3. Calculating Annualized Returns
4. Running a Regression Model 


In [ ]:
######################
###Libraries & Paths##
######################

import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import math
import statistics

## Our First Function

The "get_download_path" function allows python to find your downloads folder's filepath and use it pull open your ticker list csv file.  The output of the below cell should be a list of the tickers you inputted in the ticker list csv file.


In [ ]:
def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')

ticker_path = get_download_path()+'/Ticker_List.csv'
key = 'RGOLSJPSTGVAN4NTN4DLWJE71SU7SIH0'

tickers_file = pd.read_csv(ticker_path, header=None)
tickers = []
for ticker in tickers_file.values:
    tickers.append(ticker[0])
print(tickers)

## Manual Edits
Uncomment out the below cell and edit the list's content to replace the tickers you wish to evaluate.

In [ ]:
#tickers
# tickers = ['TSLA','AAPL']


## Defined Functions
While this code block does not run anything on its own, it holds functions that will be used later in the program.  The TD Ameritrade API contains functions that don't always output data in the most clean (useable) format, so we have developed some functions that "scrub" the output so that it returns in a better format.

In [ ]:
def get_price_hist(ticker,period,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+ticker+'/pricehistory'

    ##Define Payload
    payload = {'apikey': key,
    'periodType': 'year',
    'peirod':period,
    'frequencyType':'daily'}

    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()

    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        hist_data = pd.DataFrame(dictlist[0][1])
        hist_data['ticker'] = ticker
        hist_data['datetime'] = pd.to_datetime(hist_data['datetime'],unit='ms')
        return hist_data
    except:
        df = pd.DataFrame()
        return df
    

def get_fundamental_from_td(ticker,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    projection = 'fundamental'

    ##Define Payload
    payload = {'apikey': key,
               'symbol' : ticker,
                'projection': projection,
                }
    
    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()
    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        df = pd.DataFrame(dictlist[0][1]).T.reset_index(drop=True).iloc[0]
        return df
    except:
        print(dictlist)
        df = pd.DataFrame()
        print(ticker + " not valid.")
        return df

def scrub_fundamental_data(tickers,key):
    master = pd.DataFrame()
    count = 1
    for ticker in tickers:
        temp = get_fundamental_from_td(ticker,key)
        temp = pd.DataFrame(temp).T
        hist_data = get_annual_returns([ticker],key)
        try:
            temp.columns = ['beta', 'bookValuePerShare', 'currentRatio', 'divGrowthRate3Year',
           'dividendAmount', 'dividendDate', 'dividendPayAmount',
           'dividendPayDate', 'dividendYield', 'epsChange', 'epsChangePercentTTM',
           'epsChangeYear', 'epsTTM', 'grossMarginMRQ', 'grossMarginTTM', 'high52',
           'interestCoverage', 'low52', 'ltDebtToEquity', 'marketCap',
           'marketCapFloat', 'netProfitMarginMRQ', 'netProfitMarginTTM',
           'operatingMarginMRQ', 'operatingMarginTTM', 'pbRatio', 'pcfRatio',
           'peRatio', 'pegRatio', 'prRatio', 'quickRatio', 'returnOnAssets',
           'returnOnEquity', 'returnOnInvestment', 'revChangeIn', 'revChangeTTM',
           'revChangeYear', 'sharesOutstanding', 'shortIntDayToCover',
           'shortIntToFloat', 'ticker', 'totalDebtToCapital', 'totalDebtToEquity',
           'vol10DayAvg', 'vol1DayAvg', 'vol3MonthAvg']
            temp = pd.merge(temp, hist_data, on='ticker')
        except:
            continue
        
        master = master.append(temp).reset_index(drop=True)
        count+= 1
    return master

def scrub_price_hist(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        price_data = get_price_hist(ticker,1,key)
        master = master.append(price_data, ignore_index=True)
    return master

def get_annual_returns(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        try:
            price_data = get_price_hist(ticker,1,key).iloc[::-1]
            return_percent = price_data['close'].iloc[0] / price_data['close'].iloc[-1] -1
            df = pd.DataFrame({"ticker":ticker,  
                               "Annual_Return_Percent":return_percent},
                               index = [0])
            master = master.append(df, ignore_index=True)
        except:

            print('error with', ticker)
    return master

## Get Fundamental Data
The below cell pulls the fundamental financial data from TD Ameritrade, cleans it up into a useable format and spits it out in a nice table format using the Pandas package.

In [ ]:
scrubbed_fundamental_data=scrub_fundamental_data(tickers,key)
scrubbed_fundamental_data

## Get Annual Return Data
The below cell pulls the annual return data from TD Ameritrade, cleans it up into a useable format and spits it out in a nice table format using the Pandas package.

In [ ]:
annual_returns_df=get_annual_returns(tickers,key)
annual_returns_df

## Fundamental Data Frame Info
The next few code blocks give us information on how our data is structered, the columns it contains, the data types of the columns (string, integer, float, etc.), summary statistics, null values, etc.

In [ ]:
for col in scrubbed_fundamental_data.columns:
    try:
        scrubbed_fundamental_data[col]=scrubbed_fundamental_data[col].astype('float')
    except:
        pass

#Data columns
scrubbed_fundamental_data.columns

In [ ]:
#General data statistics
scrubbed_fundamental_data.describe()

In [ ]:
# Data Frame information (null, data type etc)
scrubbed_fundamental_data.info()

## Correlation Map

In [ ]:
# Uncorrelated features are generally more powerful predictors
colormap = plt.cm.viridis
plt.figure(figsize=(40, 40))
plt.title('Pearson Correlation of Features', y=1.05, size=14)
sns.set(font_scale=1.4)
sns.heatmap(scrubbed_fundamental_data.corr().round(2)\
            ,linewidths=0.1,vmax=1.0, square=True, cmap=colormap, \
            linecolor='white', annot=True);

## Linear Regression

In [ ]:
x = scrubbed_fundamental_data[['beta', 'pcfRatio', 'returnOnAssets','quickRatio','dividendYield']] # Independent variables
y = scrubbed_fundamental_data["Annual_Return_Percent"] # Response / Target Variable

print(x.shape, y.shape)
x=sm.add_constant(x)
result=sm.OLS(y,x).fit()
print(result.summary())

## Regression Results
The above regression results show us that by only looking five fundamental metrics, we are able to explain nearly 50% of the movement in the stocks analyzed (per R-squared).  Additionally, it is usefuly to look at the Prob (F-statistic) and Prob(Jarque-Bera) as these two statistical tests are crucial gauges of model efficacy.  Specifically, if both values are under .05, we can reasonable assume the variables used in the model are meaningful indicators.  

# TD API: A Risk Management Study
Understanding the risk of a portfolio of stocks is often times just as crucial, if not more so, than understanding its growth opportunities.  In this exercise we will examine a portfolio that has equal weights in APPL and TSLA, and attempt to gauge how much capital we are putting at risk by holding this portfolio.  To keep math simple, we will assume that the portfolio is valued at $50,000.  

## Value at Risk
Value at Risk, or simply VaR, has been around for decades, and is considered the gold standard for the benchmark of porfolio risk.  Barring any significant shifts in company health or direction, VaR can be thought of as largest paper loss an investor will see over a predefined time horizon by holding a security(s).  We will look at one month VaR, and choose a confidence level of 95% in order to have a high degree of confidence that we are in fact looking at a reasonable worst case.  While it can be beneficial to look at the 99% case, 95% will be a good start.  With our parameters defined, VaR can be calculated as follows:

VaR = Portfolio Volatility * Portfolio Value * 1.65

In [ ]:
###Define Portfolio Inputs
aapl_weight = .5
tsla_weight = .5
portfolio_value = 50000
days_into_future = 30
tickers = ['TSLA','AAPL']

In [ ]:
###Grab Price History
price_hist = scrub_price_hist(tickers,key)
price_hist = price_hist[['close','ticker','datetime']]


###Calculate Portfolio Volatility
count = 0
for ticker in tickers:
    if count == 0:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = temp
        count = 1
    else:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = pd.merge(master,temp,how = 'left',on = ['datetime'])


consol_returns = pd.DataFrame([master[tickers[0] + '_Return_%']*tsla_weight,master[tickers[1] + '_Return_%']*aapl_weight]).T
port_volatility = math.sqrt(consol_returns.cov().to_numpy().sum())*math.sqrt(days_into_future)

###Calculate Individual Security Volatilities
sec_vols = []        
for ticker in tickers:
    temp = [ticker,(math.sqrt(days_into_future)*math.sqrt(statistics.variance(master[ticker + '_Return_%'])))]
    sec_vols.append(temp)



###Compare individual security volatility to Portfolio Volatility
for i in range (0,len(tickers)):
    print(tickers[i] + ' Volatility: ',round(sec_vols[i][1]*100,2),'%')
print('Portfolio Volatility: ',round(port_volatility*100,2),'%')


In [ ]:
###Calculate VaR
VaR = portfolio_value * 1.65 * port_volatility
print('$',round(VaR,2))


## Conclusions

By holding a \\$50,000 portfolio that is equally weighted in AAPL and TSLA stock, an investor should be comfortable with a significant likelihood of losing \\$16,000+ over the course of one month.  